In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import expit
from tqdm import tqdm

from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv("train_new.csv")
test = pd.read_csv("test_new.csv")
train

,Pair,Source,Sink,NCA,Exist,CN,AA,RA,JC,PA,...,n2v_11,n2v_12,n2v_13,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20
0,"(0, 356)",0,356,14,1,7,2.899858,0.628968,0.700000,72,...,-0.441833,-0.365107,0.808580,-0.776629,-0.398342,1.080715,-0.773911,-1.598859,0.635909,-0.709417
1,"(0, 1236)",0,1236,14,1,6,2.471649,0.531746,0.428571,96,...,-0.497144,-0.305505,0.529376,-0.628553,-0.223878,0.998884,-0.651569,-1.617872,0.695837,-0.712304
2,"(356, 1236)",356,1236,14,1,7,2.812086,0.587302,0.500000,108,...,-0.600816,-0.348749,0.465387,-0.559477,-0.200811,1.017463,-0.606597,-1.639789,0.612917,-0.744530
3,"(0, 1655)",0,1655,9,1,7,2.976054,0.668651,0.466667,112,...,-0.191807,-0.108319,0.981521,-1.084302,-0.570535,0.916353,-0.617127,-1.465398,0.716408,-0.587510
4,"(0, 1797)",0,1797,4,1,7,2.899858,0.628968,0.700000,72,...,-0.302182,-0.307874,0.854235,-0.870523,-0.440440,1.122925,-0.803164,-1.635018,0.791342,-0.652107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7093258,"(4089, 4091)",4089,4091,0,0,0,0.000000,0.000000,0.000000,35,...,1.043285,0.057717,-0.351476,-0.166288,-0.301712,0.314469,0.645251,-0.759133,-0.681610,-1.176053
7093259,"(4089, 4092)",4089,4092,0,0,0,0.000000,0.000000,0.000000,25,...,1.431804,-0.714452,0.469350,-0.084874,-0.828324,0.048068,-0.529535,-1.272614,-0.734793,-0.223739
7093260,"(4090, 4091)",4090,4091,0,0,0,0.000000,0.000000,0.000000,42,...,0.758625,-0.447798,0.050211,-0.356150,-0.135803,0.239426,0.419526,-0.902752,-0.551571,-1.232052
7093261,"(4090, 4092)",4090,4092,0,0,0,0.000000,0.000000,0.000000,30,...,1.147143,-1.219967,0.871037,-0.274736,-0.662415,-0.026975,-0.755260,-1.416234,-0.604754,-0.279738


In [7]:
cond = train["Pair"].isin(list(test["Pair"]))
train.drop(train[cond].index, inplace = True)

In [8]:
train

,Pair,Source,Sink,NCA,Exist,CN,AA,RA,JC,PA,...,n2v_11,n2v_12,n2v_13,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20
0,"(0, 356)",0,356,14,1,7,2.899858,0.628968,0.700000,72,...,-0.441833,-0.365107,0.808580,-0.776629,-0.398342,1.080715,-0.773911,-1.598859,0.635909,-0.709417
1,"(0, 1236)",0,1236,14,1,6,2.471649,0.531746,0.428571,96,...,-0.497144,-0.305505,0.529376,-0.628553,-0.223878,0.998884,-0.651569,-1.617872,0.695837,-0.712304
3,"(0, 1655)",0,1655,9,1,7,2.976054,0.668651,0.466667,112,...,-0.191807,-0.108319,0.981521,-1.084302,-0.570535,0.916353,-0.617127,-1.465398,0.716408,-0.587510
4,"(0, 1797)",0,1797,4,1,7,2.899858,0.628968,0.700000,72,...,-0.302182,-0.307874,0.854235,-0.870523,-0.440440,1.122925,-0.803164,-1.635018,0.791342,-0.652107
5,"(0, 2568)",0,2568,8,1,7,2.976054,0.668651,0.466667,112,...,-0.295436,-0.377532,0.800502,-0.959644,-0.527788,0.778238,-0.678870,-1.335596,0.745900,-0.693027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7093258,"(4089, 4091)",4089,4091,0,0,0,0.000000,0.000000,0.000000,35,...,1.043285,0.057717,-0.351476,-0.166288,-0.301712,0.314469,0.645251,-0.759133,-0.681610,-1.176053
7093259,"(4089, 4092)",4089,4092,0,0,0,0.000000,0.000000,0.000000,25,...,1.431804,-0.714452,0.469350,-0.084874,-0.828324,0.048068,-0.529535,-1.272614,-0.734793,-0.223739
7093260,"(4090, 4091)",4090,4091,0,0,0,0.000000,0.000000,0.000000,42,...,0.758625,-0.447798,0.050211,-0.356150,-0.135803,0.239426,0.419526,-0.902752,-0.551571,-1.232052
7093261,"(4090, 4092)",4090,4092,0,0,0,0.000000,0.000000,0.000000,30,...,1.147143,-1.219967,0.871037,-0.274736,-0.662415,-0.026975,-0.755260,-1.416234,-0.604754,-0.279738


In [15]:
test

,Id,Source,Sink,Pair,CN,AA,RA,JC,PA,KI,...,n2v_11,n2v_12,n2v_13,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20
0,1,0,2917,"(0, 2917)",0,0.000000,0.000000,0.000000,56,-0.000552,...,-0.217625,-0.989594,0.318250,1.025849,1.072989,-0.084099,-0.696392,-0.338227,-0.114178,-0.235864
1,2,0,2956,"(0, 2956)",0,0.000000,0.000000,0.000000,24,-0.000088,...,-0.493004,-1.120674,0.378106,0.360288,1.130361,0.871391,-0.439986,-0.701735,-0.003102,-0.748156
2,3,1,4038,"(1, 4038)",0,0.000000,0.000000,0.000000,496,0.009896,...,-0.005342,0.112067,0.327012,0.046739,0.370075,0.209226,0.686743,-1.708898,-0.471961,-0.802962
3,4,2,1848,"(2, 1848)",2,1.242670,0.400000,0.080000,72,0.057307,...,0.810857,-0.774672,-0.249583,0.097737,1.224068,0.025562,-0.401903,-0.879856,-1.382236,-1.114386
4,5,3,513,"(3, 513)",0,0.000000,0.000000,0.000000,391,-0.138788,...,0.851425,-0.071756,0.850157,0.056224,0.033808,0.243672,0.351791,-0.461942,-0.055407,-0.761750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,3865,3924,"(3865, 3924)",0,0.000000,0.000000,0.000000,2,-0.000290,...,0.892352,-0.923871,0.878358,0.022944,0.345362,0.078195,0.319224,-0.527601,-0.288895,-0.225638
1996,1997,3917,4025,"(3917, 4025)",0,0.000000,0.000000,0.000000,2,0.000000,...,0.099553,-1.139272,0.974324,-0.756348,0.531660,0.827268,0.571918,-1.584577,-0.052500,-0.002654
1997,1998,3922,3947,"(3922, 3947)",0,0.000000,0.000000,0.000000,116,0.011228,...,0.394306,-0.302437,0.786825,-0.662773,0.204308,-0.346525,0.657531,-0.738892,0.063679,0.767194
1998,1999,3955,3987,"(3955, 3987)",4,1.766048,0.424368,0.053333,370,0.049551,...,0.090878,0.494751,0.378667,-0.151444,0.507886,1.182014,0.885586,-0.546440,-0.428615,-0.907595


In [18]:
y = train.Exist
X = train.drop(columns=['Pair',"Source", "Sink", "Exist"])
X

,NCA,CN,AA,RA,JC,PA,KI,PR_s1,PR_s2,n2v_1,...,n2v_11,n2v_12,n2v_13,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20
0,14,7,2.899858,0.628968,0.700000,72,0.079962,0.000220,0.000240,0.102114,...,-0.441833,-0.365107,0.808580,-0.776629,-0.398342,1.080715,-0.773911,-1.598859,0.635909,-0.709417
1,14,6,2.471649,0.531746,0.428571,96,0.075137,0.000220,0.000302,-0.031434,...,-0.497144,-0.305505,0.529376,-0.628553,-0.223878,0.998884,-0.651569,-1.617872,0.695837,-0.712304
3,9,7,2.976054,0.668651,0.466667,112,0.083302,0.000220,0.000376,0.063251,...,-0.191807,-0.108319,0.981521,-1.084302,-0.570535,0.916353,-0.617127,-1.465398,0.716408,-0.587510
4,4,7,2.899858,0.628968,0.700000,72,0.081045,0.000220,0.000245,0.106252,...,-0.302182,-0.307874,0.854235,-0.870523,-0.440440,1.122925,-0.803164,-1.635018,0.791342,-0.652107
5,8,7,2.976054,0.668651,0.466667,112,0.083444,0.000220,0.000363,0.211819,...,-0.295436,-0.377532,0.800502,-0.959644,-0.527788,0.778238,-0.678870,-1.335596,0.745900,-0.693027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7093258,0,0,0.000000,0.000000,0.000000,35,0.000327,0.000223,0.000176,1.066998,...,1.043285,0.057717,-0.351476,-0.166288,-0.301712,0.314469,0.645251,-0.759133,-0.681610,-1.176053
7093259,0,0,0.000000,0.000000,0.000000,25,0.000000,0.000223,0.000612,0.764772,...,1.431804,-0.714452,0.469350,-0.084874,-0.828324,0.048068,-0.529535,-1.272614,-0.734793,-0.223739
7093260,0,0,0.000000,0.000000,0.000000,42,0.006194,0.000201,0.000176,0.076866,...,0.758625,-0.447798,0.050211,-0.356150,-0.135803,0.239426,0.419526,-0.902752,-0.551571,-1.232052
7093261,0,0,0.000000,0.000000,0.000000,30,0.000000,0.000201,0.000612,-0.225360,...,1.147143,-1.219967,0.871037,-0.274736,-0.662415,-0.026975,-0.755260,-1.416234,-0.604754,-0.279738


In [11]:
from sklearn.linear_model import LogisticRegression

# Code adapted from Worksheet04 to calculate the coefficients
clf = LogisticRegression(penalty='none')
clf.fit(X, y)
w_sklearn = np.r_[clf.intercept_, clf.coef_.squeeze()]
print("Weights according to scikit-learn: {}".format(w_sklearn))

Weights according to scikit-learn: [-1.77596013e+01  1.03737565e+02 -5.01826776e+00  1.76376806e+01
  8.28608767e+00  3.09849646e+00  1.47641383e-03  2.43341091e+00
 -1.13128442e-03  2.72279792e-04  1.68895889e+00  2.77719402e+00
 -1.20834007e+00 -6.26919003e+00  5.54256205e+00 -8.66058142e-01
 -4.67738841e+00 -1.14650523e+01  2.43035045e+00  1.61455862e+00
 -9.20908844e+00 -2.62679426e-01  2.12443239e+00  3.29953737e+00
 -4.74711332e+00 -7.11923676e+00 -8.97982351e+00 -1.21404852e+00
  4.52087847e+00  7.58111184e+00]


/Users/abigail/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
NCA = X_test.NCA
X_test = test.drop(columns=['Id',"Source", "Sink", "Pair", "NCA"])
X_test.insert(0,'Intercept',1)
X_test.insert(1,'NCA',NCA)
X_test

,Intercept,NCA,CN,AA,RA,JC,PA,KI,PR_s1,PR_s2,...,n2v_11,n2v_12,n2v_13,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20
0,1,0,0,0.000000,0.000000,0.000000,56,-0.000552,0.000220,0.000185,...,-0.217625,-0.989594,0.318250,1.025849,1.072989,-0.084099,-0.696392,-0.338227,-0.114178,-0.235864
1,1,0,0,0.000000,0.000000,0.000000,24,-0.000088,0.000220,0.000126,...,-0.493004,-1.120674,0.378106,0.360288,1.130361,0.871391,-0.439986,-0.701735,-0.003102,-0.748156
2,1,0,0,0.000000,0.000000,0.000000,496,0.009896,0.000291,0.000891,...,-0.005342,0.112067,0.327012,0.046739,0.370075,0.209226,0.686743,-1.708898,-0.471961,-0.802962
3,1,1,2,1.242670,0.400000,0.080000,72,0.057307,0.000549,0.000115,...,0.810857,-0.774672,-0.249583,0.097737,1.224068,0.025562,-0.401903,-0.879856,-1.382236,-1.114386
4,1,0,0,0.000000,0.000000,0.000000,391,-0.138788,0.000362,0.000546,...,0.851425,-0.071756,0.850157,0.056224,0.033808,0.243672,0.351791,-0.461942,-0.055407,-0.761750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,0,0,0.000000,0.000000,0.000000,2,-0.000290,0.000071,0.000084,...,0.892352,-0.923871,0.878358,0.022944,0.345362,0.078195,0.319224,-0.527601,-0.288895,-0.225638
1996,1,0,0,0.000000,0.000000,0.000000,2,0.000000,0.000085,0.000265,...,0.099553,-1.139272,0.974324,-0.756348,0.531660,0.827268,0.571918,-1.584577,-0.052500,-0.002654
1997,1,0,0,0.000000,0.000000,0.000000,116,0.011228,0.000111,0.000745,...,0.394306,-0.302437,0.786825,-0.662773,0.204308,-0.346525,0.657531,-0.738892,0.063679,0.767194
1998,1,4,4,1.766048,0.424368,0.053333,370,0.049551,0.001486,0.000146,...,0.090878,0.494751,0.378667,-0.151444,0.507886,1.182014,0.885586,-0.546440,-0.428615,-0.907595


In [28]:
from scipy.special import expit #the logistic function
sigmoid = expit
Prob = sigmoid(X_test @ w_sklearn)
test["Predicted"] = Prob 
test

,Id,Source,Sink,Pair,CN,AA,RA,JC,PA,KI,...,n2v_12,n2v_13,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20,Predicted
0,1,0,2917,"(0, 2917)",0,0.000000,0.000000,0.000000,56,-0.000552,...,-0.989594,0.318250,1.025849,1.072989,-0.084099,-0.696392,-0.338227,-0.114178,-0.235864,2.592034e-14
1,2,0,2956,"(0, 2956)",0,0.000000,0.000000,0.000000,24,-0.000088,...,-1.120674,0.378106,0.360288,1.130361,0.871391,-0.439986,-0.701735,-0.003102,-0.748156,2.307669e-17
2,3,1,4038,"(1, 4038)",0,0.000000,0.000000,0.000000,496,0.009896,...,0.112067,0.327012,0.046739,0.370075,0.209226,0.686743,-1.708898,-0.471961,-0.802962,1.600903e-15
3,4,2,1848,"(2, 1848)",2,1.242670,0.400000,0.080000,72,0.057307,...,-0.774672,-0.249583,0.097737,1.224068,0.025562,-0.401903,-0.879856,-1.382236,-1.114386,1.000000e+00
4,5,3,513,"(3, 513)",0,0.000000,0.000000,0.000000,391,-0.138788,...,-0.071756,0.850157,0.056224,0.033808,0.243672,0.351791,-0.461942,-0.055407,-0.761750,4.419241e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,3865,3924,"(3865, 3924)",0,0.000000,0.000000,0.000000,2,-0.000290,...,-0.923871,0.878358,0.022944,0.345362,0.078195,0.319224,-0.527601,-0.288895,-0.225638,5.010807e-21
1996,1997,3917,4025,"(3917, 4025)",0,0.000000,0.000000,0.000000,2,0.000000,...,-1.139272,0.974324,-0.756348,0.531660,0.827268,0.571918,-1.584577,-0.052500,-0.002654,5.447326e-25
1997,1998,3922,3947,"(3922, 3947)",0,0.000000,0.000000,0.000000,116,0.011228,...,-0.302437,0.786825,-0.662773,0.204308,-0.346525,0.657531,-0.738892,0.063679,0.767194,2.421130e-14
1998,1999,3955,3987,"(3955, 3987)",4,1.766048,0.424368,0.053333,370,0.049551,...,0.494751,0.378667,-0.151444,0.507886,1.182014,0.885586,-0.546440,-0.428615,-0.907595,1.000000e+00


In [30]:
Pred = test[["Id", "Predicted"]]
Pred["Predicted"] = Pred["Predicted"].replace(np.nan, 0)
Pred.to_csv('LG_Prediction_with_new_features.csv', index=False)
Pred

/Users/abigail/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Id,Predicted
0,1,2.592034e-14
1,2,2.307669e-17
2,3,1.600903e-15
3,4,1.000000e+00
4,5,4.419241e-23
...,...,...
1995,1996,5.010807e-21
1996,1997,5.447326e-25
1997,1998,2.421130e-14
1998,1999,1.000000e+00
